# Explore here

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CARGA DE DATOS Y ESTRUCTURA|
--

In [4]:
#cargamos los dos csv necesasrios para la practica y los guardamos como csv local
# df1 = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv').to_csv('movies.csv')
# df2 =  pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv').to_csv('credits.csv')
#---------------------------------------------
# Carga de los csv locales
df_movies = pd.read_csv('movies.csv')
df_credits = pd.read_csv('credits.csv')
# Mostramos todas las columnas
pd.set_option('display.max_columns', None)



In [5]:
# Mostramos la estructura de los dataframes
display(df_movies.head(3))
display(df_credits.head(3))

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


,Unnamed: 0,movie_id,title,cast,crew
0,0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


---

CREACION DE BASE DE DATOS Y PREPROCESADO|
--

In [6]:
# Crear o conectar a la base de datos
conn = sqlite3.connect('movies_database.db')
# Instanciamos cursor
cursor = conn.cursor()

In [7]:
# Creamos tabla de peliculas
df_movies.to_sql('movies', conn, if_exists='replace', index=False)

4803

In [8]:
# Creamos tabla de creditos
df_credits.to_sql('credits', conn, if_exists='replace', index=False)

4803

In [9]:
# Ejecutamos la instruccion pragma para obtener datos de la tabla movies
cursor.execute('PRAGMA table_info(movies)')
columns = cursor.fetchall()
for col in columns:
    print(col)

(0, 'Unnamed: 0', 'INTEGER', 0, None, 0)
(1, 'budget', 'INTEGER', 0, None, 0)
(2, 'genres', 'TEXT', 0, None, 0)
(3, 'homepage', 'TEXT', 0, None, 0)
(4, 'id', 'INTEGER', 0, None, 0)
(5, 'keywords', 'TEXT', 0, None, 0)
(6, 'original_language', 'TEXT', 0, None, 0)
(7, 'original_title', 'TEXT', 0, None, 0)
(8, 'overview', 'TEXT', 0, None, 0)
(9, 'popularity', 'REAL', 0, None, 0)
(10, 'production_companies', 'TEXT', 0, None, 0)
(11, 'production_countries', 'TEXT', 0, None, 0)
(12, 'release_date', 'TEXT', 0, None, 0)
(13, 'revenue', 'INTEGER', 0, None, 0)
(14, 'runtime', 'REAL', 0, None, 0)
(15, 'spoken_languages', 'TEXT', 0, None, 0)
(16, 'status', 'TEXT', 0, None, 0)
(17, 'tagline', 'TEXT', 0, None, 0)
(18, 'title', 'TEXT', 0, None, 0)
(19, 'vote_average', 'REAL', 0, None, 0)
(20, 'vote_count', 'INTEGER', 0, None, 0)


In [10]:
# De nuevo ejecutamos la instruccion pragma para obtener datos, esta vez de la tabla de la tabla movies
cursor.execute('PRAGMA table_info(credits)')
columns = cursor.fetchall()
for col in columns:
    print(col)

(0, 'Unnamed: 0', 'INTEGER', 0, None, 0)
(1, 'movie_id', 'INTEGER', 0, None, 0)
(2, 'title', 'TEXT', 0, None, 0)
(3, 'cast', 'TEXT', 0, None, 0)
(4, 'crew', 'TEXT', 0, None, 0)


In [11]:
# Realizar la unión de las tablas con los datos solicitados en el enunciado
# y crear una nueva tabla con el resultado
join_query = '''
CREATE TABLE IF NOT EXISTS unified  AS
SELECT m.id, m.title, m.overview, m.genres, m.keywords, c.cast, c.crew
FROM movies m
JOIN credits c ON m.title = c.title
'''

cursor.execute(join_query)
conn.commit()


In [12]:
# importamos la tabla recien creada como dataframe
df = pd.read_sql_query('SELECT * FROM unified', conn)
display(df.head(3))

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


                            ---Tratamiento de los objetos json---

In [13]:
# Definimos funcion para convertir json a string en toda la columna
def json_to_string(dataframe, column, term):
    # importamos json para lectura de este formato
    import json
    # Lista vacia en la que se almacenan los resultados convertidos
    column_list = []
    # El bucle itera sobre cada fila de la  columna
    for index, row in dataframe.iterrows():
        # Lista vacia para almaacenar los datos de cada fila
        row_data = []
        # Verificar que no es nula o vacia 
        if row[column] and row[column] != '':
            # Carga el elemento json de esa fila
            list_ = json.loads(row[column])
            # itera sobre los diccionarios obtenidos despues de cargar el objeto json y..
            for dic in list_:
                #..extrae un termino especifico y lo añade a la lista vacia row_data
                row_data.append(dic[term])
            # este ultimo bucle for itera sobre la lista temporal row_data para eliminar comas y espacios
            for i in range(len(row_data)):
                row_data[i] = row_data[i].replace(',', '').replace(' ', '')
            # Se añade ese valor ya formateado a la lista final
            column_list.append(' '.join(row_data))
    # La funcion devuelve la lista final
    return column_list

In [14]:
# Llamamos a la fucnion anterior para formatear las columnas necesarias
df['genres'] = json_to_string(df, 'genres', 'name')
df['keywords'] = json_to_string(df, 'keywords', 'name')

In [15]:
# Definimos bucle for para extraer el valor "director" de la clumna 'cast'
import json

# Creamos una lista vacía para almacenar los resultados
directors = []
# para cada dato en la columna..
for list_str in df['crew']:
    # Convertir cadena json en una lista de diccionarios
    list_ = json.loads(list_str)
    # Variable que almacena el nombre del direfctor
    director = ''
    # recorremos cada diccionario obtenido al leer el objeto json..
    for dic in list_:
        # ..y si la clave de job es Director
        if dic['job'] == 'Director':
            # cogemos el valor de la clave 'name' y eliminamos comas y espacios 
            director = dic['name'].replace(',', '').replace(' ', '')
    # Finalmente añadimos a la lista directors        
    directors.append(''.join(director))

In [16]:
# Creamos columna con el nombre del director
df['director'] = directors

In [17]:
# Muestro cast para ver la estructura 
display(df['cast'][0])

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [173]:
import json
# Bucle para obtener los tres primeros miembros del casting
cast_list = []
for list_str in df['cast']:
    row_list = []
    list_ = json.loads(list_str)
    for dic in list_[:3]:
        row_list.append(dic['name'].replace(' ', '').replace(',', '').replace('. ', ''))
    cast_list.append(' '.join(row_list))

                            --- Terminamos con los objetos json ---

In [174]:
# definbimnos la columna cast y eliminamos la columna crew

df['cast'] = cast_list

df.drop('crew', axis=1, inplace=True)

In [176]:
# Finalmente, como especifica el enunciado, creamos un dataframe con una unica columna que une todos los datos relevantes
df_final = pd.DataFrame()
df_final['tags'] = df['overview'] + ' ' + df['genres'] + ' ' + df['keywords'] + ' ' + df['cast'] + ' ' + df['director']


In [177]:
# Muestra de las dos primeras filas para ver el resultado
for tag in df_final['tags'][:2]:
    print(tag)    

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron
Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. Adventure Fantasy Action ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski


In [178]:
# Eliminamos comas y puntos de todo el dataframe y nos aseguramos de que todo esta en forma de string y que conservamos todos los datos
df_final['tags'] = df_final['tags'].apply(lambda x: str(x).replace(',', '').replace('.', ''))
df_final['tags'] = df_final['tags'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    4809 non-null   object
dtypes: object(1)
memory usage: 37.7+ KB


---

VECTORIZACION Y MATRIZ DE SIMILITUD|
--

In [179]:
# Crear una instancia de VECTORIZADOR
vectorizer = TfidfVectorizer()

# Ajustar el vectorizador al texto y transformar el texto en vectores
vectors = vectorizer.fit_transform(df_final['tags'])

In [180]:
# Crear matriz de similitud
similarity = cosine_similarity(vectors)

---

FUNCION PARA CONSULTAS|
--

(EN BASE A ELLA SE HARA LA PRACTICA DE STREAMLIT)


In [193]:
def recommend(titulo_parcial):
    # Convertimos el input a minúsculas
    titulo_parcial = titulo_parcial.lower()
    # Buscamos coincidencias parciales en los títulos.
    titles = df['title'].str.lower()
    coincidencias = titles[titles.str.contains(titulo_parcial)]
    # Definimos lo que ocurre si se encuentran coincidencias con el titulo
    if not coincidencias.empty:
        print(f"Hay {len(coincidencias)} películas que contienen '{titulo_parcial}':")
        # Para cada indice de las coincidencias encontradas
        for movie_index in coincidencias.index:
            # Extrae las distancias (similitudes) de cada película encontrada.
            distances = similarity[movie_index]
            # generamos una lista con las 5 peliculas mas parecidas eliminando la primera que es ella misma
            movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
            # imprimimos el resultado // df.loc encuentra el titulo en el dataframe original en base al indice generado con movie_index
            print(f"\nRecomendaciones basadas en '{df.loc[movie_index, 'title']}':")
            # Imprime los títulos de las películas más similares.
            for i in movie_list:
                print(df.iloc[i[0]]['title'])
    else:
        print(f"La película '{titulo_parcial}' no se encuentra en nuestro directorio.")


EJEMPLO DE USO|
--

In [196]:
recommend('when')

Hay 10 películas que contienen 'when':

Recomendaciones basadas en 'What to Expect When You're Expecting':
Mother and Child
L!fe Happens
You Will Meet a Tall Dark Stranger
I Don't Know How She Does It
About Schmidt

Recomendaciones basadas en 'Ace Ventura: When Nature Calls':
Rapa Nui
Sinbad: Legend of the Seven Seas
The Golden Child
Princess Mononoke
Caravans

Recomendaciones basadas en 'Dumb and Dumberer: When Harry Met Lloyd':
Dogtown and Z-Boys
Lords of Dogtown
Recess: School's Out
Back to the Future Part II
Harry Potter and the Philosopher's Stone

Recomendaciones basadas en 'When Harry Met Sally...':
The Story of Us
The Weather Man
Sex Tape
Celeste & Jesse Forever
A Lot Like Love

Recomendaciones basadas en 'When a Stranger Calls':
Mighty Joe Young
Jack and Jill
Scream 4
Phone Booth
The Whole Ten Yards

Recomendaciones basadas en 'When the Game Stands Tall':
We Are Marshall
Coach Carter
The Slaughter Rule
Remember the Titans
The 5th Quarter

Recomendaciones basadas en 'Things to 